# (4) Responses and Feedback

Here, we learn how to add a feedback stimulus indicating a correct or false response.

## Installing sweetbean

In [ ]:
!pip install sweetbean

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Now, we are looking to incorporate responses from the participant and response feedback to the experiment. Specifically, we would like the participant to press the key `f` using their left index finger when the ink color of the color word is red. Similarly, we would like the participant to press the key `j` with their right index finger when the color is green.

You can practice your SweetBean skills by implementing these instructions in a block (if you feel comfortable creating text stimuli you can skip this part and use the instructions block in the solution):

In [ ]:
# Enter your code here:

### Solution

In [ ]:
# imports
from sweetbean.stimulus import TextStimulus
from sweetbean.sequence import Block

# Creating the Instructions
welcome = TextStimulus(text="Welcome to our experiment.<br>Here, you will have to react to the ink color of a color word.<br>Press SPACE to continue", choices=[' '])
instruction_red = TextStimulus(text="If the ink color is <b>red<b>,<br>press <b>F<b> with your left index finger as fast as possible.<br>Press F to continue", choices=['f'])
instruction_green = TextStimulus(text="If the ink color is <b>green<b>,<br>press <b>J<b> with your right index finger as fast as possible.<br>Press J to continue", choices=['j'])
instructions_end = TextStimulus(text="The experiment will start now.<br>React as fast an as accurate as possible.<br>Remember:<br>React to the ink color not the meaning of the word.<br>Ress SPACE to continus", choices=[' '])

# Creating the stimulus sequence
instructions_sequence = [welcome, instruction_red, instruction_green, instructions_end]

# Creating the block
instructions_block = Block(instructions_sequence)

## Creating a timeline

In [ ]:
timeline = [
    {'color': 'red', 'word': 'RED'},
    {'color': 'green', 'word': 'GREEN'},
    {'color': 'green', 'word': 'RED'},
    {'color': 'red', 'word': 'GREEN'},
]

## Declare TimelineVariables
Do you remember how to declare the timeline variables(if you feel comfortable creating timeline variables, you can skip this part and use the solution)?

In [ ]:
# Enter your code here:

### Solution

In [ ]:
# import the functionality from sweetbean
from sweetbean.parameter import TimelineVariable

## declare the timeline variables

# color: The name has to be color (it is the name in the timeline), and it has the levels red and green
color = TimelineVariable(name="color", levels=["red", "green"])

# word: The name has to be word (it is the name in the timeline), and it has the levels RED and GREEN
word = TimelineVariable(name="word", levels=["RED", "GREEN"])


## Creating derived variables
In SweetBean, in addition to static parameters and timeline parameters, we can also use dervived parameters. These are parameters, that derive from other features of the trial. Here, we want to derive the correct key from the ink color of the stimulus. Remember: `f` for red color words and `j` for green color words.

### Defining the functions
To define the various levels of the derived parameter, we need to create functions (known as predicates) for each level. These functions should accept the features that the derived parameter depends on and should return `True` when the conditions for the specific level of the derived parameter are met.

For example, the function for the level `f` should return `True` if the color is red, and `False` in all other cases. Similarly, the function for the level `j` should return `True` if the color is green, and `False` in all other cases.

In [ ]:
# defining the predicate for the f-level of the "correct response" parameter
def is_correct_f(color):
  return color == 'red'

# defining the predicate for the j-level of the "correct response" paramater
def is_correct_j(color):
  return color == 'green'

Some people might have trouble understanding the code above. A equivalent writing with an if and else statement is:
```
def is_correct_f(color):
  if color == 'red':
    return True
  else:
    return False

def is_correct_j(color):
  if color == 'green':
    return True
  else:
    return False
```
<details><summary>Optional: Python is whitespace sensitive!</summary>In contrast to many other programming languages, Python is whitespace sensitive. Whitespaces in Python do matter and are used to block the code, for example to define functions, in loops or blocks of if-else statements. In that sense they are equivalent to the curly brackets '{}' in Java, JavaScript, C++ and many other programming languages.
</details>


## Creating the derived levels
With the predicates defined, we can now define the corresponding levels. The first argument of the derived level constructor is it's value, the second is the predicate and the third is a list of the parameters it depends on.

In [ ]:
# importing the functionality
from sweetbean.parameter import DerivedLevel

# declare the f level
correct_response_f = DerivedLevel(value='f', predicate=is_correct_f, factors=[color])

# declare the j level
correct_response_j = DerivedLevel('j', is_correct_j, [color])

## Create the derived Parameter
Now, that we declared the levels of the parameter, we can declare the parameter itself.

In [ ]:
# importing the functionality
from sweetbean.parameter import DerivedParameter

# declare the "correct response" parameter
correct_response = DerivedParameter(name='correct_response', levels=[correct_response_f, correct_response_j])

## Create the stimulus
The derived parameter can be used in the creation of the stimulus just as a static or a timeline parameter. Here, we introduce another feature of the text stimulus: We can provide a correct_key parameter. This correct_key parameter can be used for data analysis later, but it can also be used to provide feedback (we will see how this is used later in this tutorial)

In [ ]:
# imports
from sweetbean.stimulus import TextStimulus

# declaring the stimulus
stroop = TextStimulus(duration=2500, text=word, color=color, choices=['j', 'f'], correct_key=correct_response)

This is a stimulus that shows a color word in the color and with the word provided by the timeline. The stimulus is shown for 2500ms or till a response is given. The user can press either `j` or `f`.

<details><summary>Comprehension question: Can you think of another way to get the correct_key parameter that doesnt' depend on a derived paramter?</summary>Here, we used the derived paramter to demonstrate how it is used. But we could also simply add the keys to the timeline and use a timeline varibale insted. Often times, there are multiple ways of implementing a desired experiment.</details>

## Adding feedback
To add feedback, we introduce yet another type of variable. The data-variable. At the end of each stimulus, data for this stimulus is stored in a container. This data contains stimulus features like color, word and duration, but also user input like key presses. With the data variable, we can access this container to create adaptive stimuli like feedback.


## Creating the data variable
We create a data variable for the correct-data of a trial. This is a special data point, that can be accessed in stimuli that have the correct_key parameter. A comprehensive list of data points that can be accessed will be made available in the documentation (It is mainly the data that gets stored via <a href="https://www.jspsych.org/7.3/">jsPsych</a>.

In [ ]:
# import
from sweetbean.parameter import DataVariable

# declare the data variable
correct = DataVariable('correct', [True, False])

## Defining the function
In the same way we can access timeline variables in predicate functions, we can also access data variables.

In [ ]:
# positive feedback after correct responses (remember the correct data variable has boolean levels itself)
def is_positive_feedback(correct):
  return correct

# negative feedback after incorrect responses
def is_negative_feedback(correct):
  return not correct

## Creating the derived levels
Now we create the levels for the text parameter of the feedback stimulus.

Note: We provide an additional argument to this constructor. The purpose of this argument is to access the data variable of the stimulus that came before the current stimulus, which is the stroop stimulus in this case.

The number `1` provided at the end of the constructor indicates that we want to access the data variable of the stimulus that occurred one stimulus back in time, which is the stroop stimulus. If we were to include an additional onset between the stroop and the feedback stimuli, we could insert a blank screen and then access the correct-data variable of the stimulus that occurred two stimuli back by providing a `2` as the last argument of the constructor.

In [ ]:
positive_word_feedback = DerivedLevel('correct', is_positive_feedback, [correct], 1)
negative_word_feedback = DerivedLevel('false', is_negative_feedback, [correct], 1)

## Creating parameter
We create the word parameter for the feedback stimulus

In [ ]:
feedback_word = DerivedParameter('feedback_word', [positive_word_feedback, negative_word_feedback])

<details><summary>Comprehension question: Can you think of another way to get the feeback word parameter that doesnt' depend on a derived paramter?</summary>Here, we can not use a timeline, since this is an adaptive parameter. We have to use the derived parameter and the data variable since we have use information derived from user input.</details>

## Create a parameter for the color of the feedback.
Maybe we also want to change the color of the feedback (green for positive and red for negative feedback). Can you create the derived parameter?

In [ ]:
# Enter your code:
# ...
feedback_color = None

### Solution

In [ ]:
# create the levels
positive_color_feedback = DerivedLevel('green', is_positive_feedback, [correct], 1)
negative_color_feedback = DerivedLevel('red', is_negative_feedback, [correct], 1)
# create the parameter
feedback_color = DerivedParameter('feedback_color', [positive_color_feedback, negative_color_feedback])

## Creating the feedback stimulus

In [ ]:
feedback = TextStimulus(duration=1000, text=feedback_word,color=feedback_color)

## Adding a fixation cross, creating the block, creating the experiment and exporting the html

You can practice your SweetBean skills by adding a fixation cross and finishing experiment (Remember there are two blocks now, if you feel comfortable creating blocks and experiments you can skip this part and use the code in the solution):

In [ ]:
# Enter your code:

### Solution

In [ ]:
# import the functionality from sweetbean to create experiments
from sweetbean.sequence import Block, Experiment

# fixation stimulus
fixation = TextStimulus(800, '+')

# create a stimulus sequence
stimulus_sequence = [fixation, stroop, feedback]

# create the trial block
trial_block = Block(stimulus_sequence, timeline)

# create the experiment from the two blocks
experiment = Experiment([instructions_block, trial_block])

# export to the html file
experiment.to_html('index.html')
